# Detectron2 - Archeo dataset

## Prepare env.

Make sure Detectron2 is installed before preparing env.

In [ ]:
#check torch and gcc
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
#import standard lib.
import os
import numpy as np
import cv2
import random
import json
import random
import matplotlib.pyplot as plt
import time
import logging
import torch
import time
import datetime
import copy

# import detectron2 and utilities
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.engine.hooks import HookBase
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.logger import log_every_n_seconds
import detectron2.utils.comm as comm
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog, DatasetMapper, build_detection_test_loader
from detectron2.data import transforms as T
from detectron2.data import build
from detectron2.data import detection_utils as utils
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.evaluation import inference_context

## Configure project

In [ ]:
#Dataset home dir containing dataset
ARCHEO_DIR = "./datasets/example_dataset"

#Train and Validation folder in ARCHEO_DIR
TRAIN_DIR = "train"
VALIDATION_DIR = "test"

#Output dir to store model
OUTPUT_DIR = "./outputs"

### Registering the Dataset

Annotations must be in COCO format (eg. via_region_data.json)

In [ ]:
def get_archeo_dicts(img_dir):
    print (img_dir)
    assert (os.path.isfile(os.path.join(img_dir, "via_region_data.json")))
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for _, anno in annos.items():
            # assert not anno["region_attributes"] #If this is not taken into account is monoclass
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in [TRAIN_DIR, VALIDATION_DIR]:
    DatasetCatalog.register("archeo_" + d, lambda d=d: get_archeo_dicts(os.path.join(ARCHEO_DIR, d)))
    MetadataCatalog.get("archeo_" + d).set(thing_classes=["archeo"])
    print (MetadataCatalog.get("archeo_" + d))

train_dataset_meta = MetadataCatalog.get("archeo_" + TRAIN_DIR)
train_dataset = DatasetCatalog.get("archeo_" + TRAIN_DIR)
test_dataset_meta = MetadataCatalog.get("archeo_" + VALIDATION_DIR)
test_dataset = DatasetCatalog.get("archeo_" + VALIDATION_DIR)

### Configure model

In [ ]:
cfg = get_cfg()

# get config for pre-trained model (Mask R-CNN, Resnet-101)
# https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
# get pre-trained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")

# Number of data loading threads
cfg.DATALOADER.NUM_WORKERS = 2
# To use cropping while data augmentation
cfg.INPUT.CROP.ENABLED = True
# Input image format
cfg.INPUT.FORMAT = 'RGB'
# Number of class
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# Input image size
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MIN_SIZE_TRAIN = 512
#Specify anchor sizes
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256, 512]]

#Configuration of dataset and output
cfg.DATASETS.TRAIN = ("archeo_" + TRAIN_DIR,)
cfg.DATASETS.TEST = ("archeo_" + VALIDATION_DIR,)

#Creating an output dir 
cfg.OUTPUT_DIR=OUTPUT_DIR

#Configuration of solver
cfg.SOLVER.IMS_PER_BATCH = 2
#Iterations in which the learning rate will be multiplied by gamma factor.
cfg.SOLVER.STEPS = (500, 1000, 1500)
#Learning rate - This learning rate will be used ONLY once the warmup period has ended
cfg.SOLVER.BASE_LR = 0.002
#Maximum iterations
#The maximum of iterations is calculated by multiplying the amount of epochs times the amount of images times the images per batch
cfg.SOLVER.MAX_ITER = 2000
#Warmup iteration before adaptive LR
cfg.SOLVER.WARMUP_ITERS = 100

#Freeze all but heads
cfg.FREEZE_AT = 4

#Test performance on the DATASETS.TEST every EVAL_PERIOD
cfg.TEST.EVAL_PERIOD = 100

### Configure data augmentation

In [ ]:
def mapper(dataset_dict):
    # Implement a mapper with data augmentation
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format=cfg.INPUT.FORMAT)

    image, transforms = T.apply_transform_gens([
        T.RandomFlip(prob=0.50, horizontal=True, vertical=False),
        T.RandomFlip(prob=0.50, horizontal=False, vertical=True),
        T.RandomApply(T.RandomCrop(crop_type="relative_range", crop_size=(0.8, 0.8)), 
                       prob=0.50),       
        T.RandomApply(T.RandomRotation(angle=[-30,30], expand=True, center=None, sample_style="range", interp=None), 
                      prob=0.50)
    ], image)
    
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

## Train model

In [ ]:
class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks
    
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        return build.build_detection_test_loader(cfg, dataset_name, mapper=DatasetMapper(cfg, False))

    @classmethod
    def build_train_loader(cls, cfg):
        return build.build_detection_train_loader(cfg, mapper=mapper) ## To apply transformations

### Run training

In [ ]:
trainer = MyTrainer(cfg)

In [ ]:
trainer.resume_or_load(resume=False)
trainer.train()

## Evaluating Model

In [ ]:
#Load fine-tuned model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#Testing threshold for this model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
#Config predictor
predictor = DefaultPredictor(cfg) 
#Dataset for evaluation
#dataset = test_dataset_meta.name

### Stats. results

In [ ]:
#configure evaluation
evaluator = COCOEvaluator(test_dataset_meta.name, cfg, False, output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, test_dataset_meta.name)
inference_on_dataset(trainer.model, val_loader, evaluator)

### Visual results (detected segment)

In [ ]:
total_plots = len(test_dataset)
cols = 2

rows = total_plots // cols 
rows += total_plots % cols
pos = range(1,total_plots + 1)

fig = plt.figure(1, figsize=(10*cols, 10*rows))

for i, d in enumerate(test_dataset):
    
    #load image
    im = cv2.imread(d["file_name"])
    
    ax = fig.add_subplot(rows,cols,pos[i])
    #load image
    im = cv2.imread(d["file_name"])

    #run prediction
    outputs = predictor(im)

    #get results prediction
    v = Visualizer(im[:, :, ::-1],
                   metadata=test_dataset_meta, 
                   scale=1
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    ax.set_title('Prediction: ' + d["file_name"].split('/')[-1])
    ax.imshow(out.get_image())
    ax.set_axis_off()

plt.show()